<a href="https://colab.research.google.com/github/Nadiyapathan/pyspark/blob/main/Tungsten_%26_Catalyst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from pyspark.sql import SparkSession

Spark=SparkSession.builder.appName("Pyspark").getOrCreate()

In [14]:
data=[("nadi",32),("Jabi",39),("Ayesha",6),("Ayaan",3)]
columns=["Name","Age"]
df = Spark.createDataFrame(data,columns)
df.show()

+------+---+
|  Name|Age|
+------+---+
|  nadi| 32|
|  Jabi| 39|
|Ayesha|  6|
| Ayaan|  3|
+------+---+



In [16]:
from pyspark.sql.functions import col

large_df = Spark.range(1, 1000000)
result_df=large_df.select((col("id")*2).alias("double_id"))
result_df.show(10)

+---------+
|double_id|
+---------+
|        2|
|        4|
|        6|
|        8|
|       10|
|       12|
|       14|
|       16|
|       18|
|       20|
+---------+
only showing top 10 rows

